# 240910
## 초안

# 20240913
## 1. wk,rk : output에 이쁘게 정리
## 2. trocs : output에 이쁘게 정리


# 20240914 
## 1. nau파일 불러오기
### vlookup -> 매핑방식 으로 변경
### DataFrame -> List 구조로 변경 (시간단축)

# 20240914 (2)
### MULTI LOT REGRESSION :  gpt-o1이 살짝 수정

# 20240914 (3)
### pred(fit, res)를 RawData-1 시트에 추가.

# 20240914 (4)
### wf좌표 추가

# 20240914 (5)
### Fit 이름 수정.  Y_dx_pred -> pred_x



In [1]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl



# nau 파일이 있는 폴더 경로
folder_path = 'C:/py_data/nau'

# 추출할 컬럼 위치 설정 (예: 첫 번째 열은 0, 두 번째 열은 1로 인덱스 시작)
columns_to_extract = [0, 1, 2, 3, 4, 5, 6, 7]  # Wafer, TEST, DieX, DieY, X_reg, Y_reg, MRC_X, MRC_Y의 열 위치


# 결과를 담을 리스트 생성
combined_rawdata_list = []
combined_trocs_input_list = []
mrc_data_list = []

# 폴더 내 모든 nau 파일에 대해 반복
for file_name in os.listdir(folder_path):
    if file_name.endswith('.nau'):
        file_path = os.path.join(folder_path, file_name)
        
        # 필요한 시트만 읽기
        rawdata_file = pd.read_excel(file_path, sheet_name='RawData-1')
        trocs_input_file = pd.read_excel(file_path, sheet_name='Trocs Input')

        ''' RAW DATA '''
        # 지정된 열 추출
        extracted_data_raw = rawdata_file.iloc[:, columns_to_extract].copy()

        # 'STEPSEQ' 및 'LOT_ID' 값 추출
        stepseq_value_raw = rawdata_file.iloc[0, 13]
        lot_id_value_raw = rawdata_file.columns[13]

        # 새로운 컬럼 추가
        extracted_data_raw['STEPSEQ'] = stepseq_value_raw
        extracted_data_raw['LOT_ID'] = lot_id_value_raw

        # 컬럼 순서 재조정
        cols_raw = ['STEPSEQ', 'LOT_ID'] + extracted_data_raw.columns[:-2].tolist()
        extracted_data_raw = extracted_data_raw[cols_raw]

        # 추가 정보 추출 및 컬럼 추가
        extracted_data_raw['STEP_PITCH_X'] = rawdata_file.iloc[6, 13]
        extracted_data_raw['STEP_PITCH_Y'] = rawdata_file.iloc[7, 13]
        extracted_data_raw['MAP_SHIFT_X'] = rawdata_file.iloc[8, 13]
        extracted_data_raw['MAP_SHIFT_Y'] = rawdata_file.iloc[9, 13]

        # 'coordinate_X', 'coordinate_Y' 매핑
        coord_map = rawdata_file[['Test No', 'coordinate_X', 'coordinate_Y']].drop_duplicates(subset='Test No').set_index('Test No')
        extracted_data_raw['coordinate_X'] = extracted_data_raw['TEST'].map(coord_map['coordinate_X'])
        extracted_data_raw['coordinate_Y'] = extracted_data_raw['TEST'].map(coord_map['coordinate_Y'])

        # 'wf_x' 및 'wf_y' 계산
        extracted_data_raw['wf_x'] = (
            extracted_data_raw['DieX'] * extracted_data_raw['STEP_PITCH_X'] +
            extracted_data_raw['MAP_SHIFT_X'] + extracted_data_raw['coordinate_X']
        )
        extracted_data_raw['wf_y'] = (
            extracted_data_raw['DieY'] * extracted_data_raw['STEP_PITCH_Y'] +
            extracted_data_raw['MAP_SHIFT_Y'] + extracted_data_raw['coordinate_Y']
        )

         # 컬럼 순서 재조정
        cols_order = [
            'STEPSEQ', 'LOT_ID', 'Wafer', 'TEST', 'DieX', 'DieY',
            'X_reg', 'Y_reg', 'MRC_X', 'MRC_Y', 'STEP_PITCH_X', 'STEP_PITCH_Y',
            'MAP_SHIFT_X', 'MAP_SHIFT_Y', 'coordinate_X', 'coordinate_Y', 'wf_x', 'wf_y'
        ]
        extracted_data_raw = extracted_data_raw[cols_order]
      
        # 리스트에 추가
        combined_rawdata_list.append(extracted_data_raw)

        ''' TROCS INPUT '''
        # 'LOT_ID' 컬럼 추가 및 순서 재조정
        trocs_input_file['LOT_ID'] = lot_id_value_raw
        cols_trocs = ['LOT_ID'] + trocs_input_file.columns[:-1].tolist()
        trocs_input_file = trocs_input_file[cols_trocs]

        # 리스트에 추가
        combined_trocs_input_list.append(trocs_input_file)


        ''' MRC '''
        # MRC 데이터 추출을 위해 'RawData-1' 시트를 header=None으로 다시 읽음
        rawdata_file_no_header = pd.read_excel(file_path, sheet_name='RawData-1', header=None)

        # MRC 데이터 추출
        mrc_part1 = rawdata_file_no_header.iloc[0:20, 15:17]
        mrc_part2 = rawdata_file_no_header.iloc[22:40, 15:17]
        mrc_part = pd.concat([mrc_part1, mrc_part2], ignore_index=True)
        mrc_part.columns = ['K PARA', 'GPM']
        mrc_part['STEPSEQ'] = stepseq_value_raw
        mrc_part['LOT_ID'] = lot_id_value_raw

        # 리스트에 추가
        mrc_data_list.append(mrc_part)

# 리스트를 데이터프레임으로 병합
combined_rawdata = pd.concat(combined_rawdata_list, ignore_index=True)
combined_trocs_input = pd.concat(combined_trocs_input_list, ignore_index=True)
mrc_data = pd.concat(mrc_data_list, ignore_index=True)

# 최종 데이터를 엑셀 파일로 저장
with pd.ExcelWriter('output.xlsx') as writer:
    combined_rawdata.to_excel(writer, sheet_name='RawData-1', index=False)
    combined_trocs_input.to_excel(writer, sheet_name='Trocs Input', index=False)
    mrc_data.to_excel(writer, sheet_name='MRC', index=False)
    







################################### MULTI LOT REGRESSION #####################################################################


# 데이터 불러오기
df = pd.read_excel("output.xlsx", sheet_name='RawData-1')

# LOT_ID별로 그룹화
grouped = df.groupby('LOT_ID')

# 회귀분석 결과를 저장할 리스트
wkrk_results = []

# 각 그룹에 대해 처리
for lot_id, group in grouped:
    die_x = group['DieX']
    die_y = group['DieY']
    step_pitch_x = group['STEP_PITCH_X']
    step_pitch_y = group['STEP_PITCH_Y']
    map_shift_x = group['MAP_SHIFT_X']
    map_shift_y = group['MAP_SHIFT_Y']
    field_x = group['coordinate_X']
    field_y = group['coordinate_Y']
    
    # 좌표 계산
    x = die_x * step_pitch_x + map_shift_x
    y = die_y * step_pitch_y + map_shift_y
    rx = field_x
    ry = field_y

    # X_dx, X_dy 행렬 구성
    X_dx = np.vstack([
        np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
        rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
    ]).T
    X_dy = np.vstack([
        np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
        ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
    ]).T

    # 종속변수
    Y_dx = group['X_reg']
    Y_dy = group['Y_reg']

    # 최소자승법으로 계수 계산
    coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]
    coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]

    # 결과 저장
    wkrk_results.append(pd.DataFrame({
        'LOT_ID': [lot_id],
        'WK1': [coeff_dx[0]],
        'WK2': [coeff_dy[0]],
        'WK3': [coeff_dx[1]],
        'WK4': [coeff_dy[1]],
        'WK5': [coeff_dx[2]],
        'WK6': [coeff_dy[2]],
        'WK7': [coeff_dx[3]],
        'WK8': [coeff_dy[3]],
        'WK9': [coeff_dx[4]],
        'WK10': [coeff_dy[4]],
        'WK11': [coeff_dx[5]],
        'WK12': [coeff_dy[5]],
        'WK13': [coeff_dx[6]],
        'WK14': [coeff_dy[6]],
        'WK15': [coeff_dx[7]],
        'WK16': [coeff_dy[7]],
        'WK17': [coeff_dx[8]],
        'WK18': [coeff_dy[8]],
        'WK19': [coeff_dx[9]],
        'WK20': [coeff_dy[9]],
        'RK1': [0],
        'RK2': [0],
        'RK3': [coeff_dx[10]],
        'RK4': [coeff_dy[10]],
        'RK5': [coeff_dx[11]],
        'RK6': [coeff_dy[11]],
        'RK7': [coeff_dx[12]],
        'RK8': [coeff_dy[12]],
        'RK9': [coeff_dx[13]],
        'RK10': [coeff_dy[13]],
        'RK11': [coeff_dx[14]],
        'RK12': [coeff_dy[14]],
        'RK13': [coeff_dx[15]],
        'RK14': [coeff_dy[15]],
        'RK15': [coeff_dx[16]],
        'RK16': [coeff_dy[16]],
        'RK17': [coeff_dx[17]],
        'RK18': [coeff_dy[17]],
        'RK19': [coeff_dx[18]],
        'RK20': [0],
    }))





# 결과 병합 및 엑셀 저장
combined_results = pd.concat(wkrk_results, ignore_index=True)
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    combined_results.to_excel(writer, sheet_name='WKRK', index=False)



################################### MULTI LOT FITTING, RESIDUAL #####################################################################


# 데이터 불러오기
df = pd.read_excel('output.xlsx', sheet_name='RawData-1')
coeff_df = pd.read_excel('output.xlsx', sheet_name='WKRK')

# LOT_ID별로 그룹화
grouped = df.groupby('LOT_ID')

# 예측 결과를 저장할 리스트
predictions_list = []

for lot_id, group in grouped:
    die_x = group['DieX']
    die_y = group['DieY']
    step_pitch_x = group['STEP_PITCH_X']
    step_pitch_y = group['STEP_PITCH_Y']
    map_shift_x = group['MAP_SHIFT_X']
    map_shift_y = group['MAP_SHIFT_Y']
    field_x = group['coordinate_X']
    field_y = group['coordinate_Y']
    
    x = die_x * step_pitch_x + map_shift_x
    y = die_y * step_pitch_y + map_shift_y
    rx = field_x
    ry = field_y

    X_dx = np.vstack([
        np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
        rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
    ]).T
    X_dy = np.vstack([
        np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
        ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
    ]).T

    # 해당 LOT_ID의 계수 추출
    coeffs = coeff_df[coeff_df['LOT_ID'] == lot_id].iloc[0]
    coeff_dx = coeffs[['WK1','WK3','WK5','WK7','WK9','WK11','WK13','WK15','WK17','WK19','RK3','RK5','RK7','RK9','RK11','RK13','RK15','RK17','RK19']].values.astype(float)
    coeff_dy = coeffs[['WK2','WK4','WK6','WK8','WK10','WK12','WK14','WK16','WK18','WK20','RK4','RK6','RK8','RK10','RK12','RK14','RK16','RK18']].values.astype(float)

    # 예측값 계산
    pred_x = X_dx.dot(coeff_dx)
    pred_y = X_dy.dot(coeff_dy)

    # 잔차 계산
    residual_x = group['X_reg'] - pred_x
    residual_y = group['Y_reg'] - pred_y

    # 결과 저장
    predictions_list.append(pd.DataFrame({
        'pred_x': pred_x,
        'pred_y': pred_y,
        'residual_x': residual_x,
        'residual_y': residual_y,
    }))

# 예측 결과 병합 및 엑셀 저장
df_predictions = pd.concat(predictions_list, ignore_index=True)
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    df_predictions.to_excel(writer, sheet_name='Predictions', index=False)
    

# 엑셀 파일 불러오기
file_path = 'C:/vscode/MODULE/output.xlsx'
excel_file = pd.ExcelFile(file_path)

# 기존 "rawdata-1" 시트 데이터 불러오기
df_rawdata = pd.read_excel(excel_file, sheet_name='RawData-1')

# 예측 데이터 df_predictions 불러오기 (예시)
df_predictions = pd.read_excel(excel_file, sheet_name='Predictions')

# 기존 데이터에 df_predictions의 열을 새롭게 추가하며 병합
df_combined = pd.concat([df_rawdata, df_predictions], axis=1)

# 엑셀 파일의 기존 시트에 덮어쓰기
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_combined.to_excel(writer, sheet_name='RawData-1', index=False)






############################ MULTI LOT CPE ############################################################################



# 엑셀 파일에서 데이터 불러오기 (실제 파일 경로로 변경 필요)
df = pd.read_excel("C:/vscode/MODULE/output.xlsx", "RawData-1")

# 'shot' 별로 데이터를 그룹화 (고유한 die_x, die_y 조합)
grouped = df.groupby(['LOT_ID', 'DieX', 'DieY'])

# 회귀분석 결과를 저장할 리스트 (딕셔너리 객체들로 구성됨)
shot_regression_results = []  # 딕셔너리가 아닌 리스트로 선언



for (lot_id, die_x, die_y), group in grouped:
    # 독립변수와 종속변수 설정 ('coordinate_X', 'coordinate_Y'를 독립변수로 사용)
    X = group[['coordinate_X', 'coordinate_Y']]
    Yx = group['residual_x']
    Yy = group['residual_y']

    # 독립변수 배열 구성 및 최소자승법으로 회귀 계수 구하기
    X_dx = np.vstack([np.ones(len(X)), X['coordinate_X']/10**6, X['coordinate_Y']/10**6]).T
    coeff_dx = np.linalg.lstsq(X_dx, Yx, rcond=None)[0]
    
    X_dy = np.vstack([np.ones(len(X)), X['coordinate_Y']/10**6, X['coordinate_X']/10**6]).T
    coeff_dy = np.linalg.lstsq(X_dy, Yy, rcond=None)[0]

    # 회귀분석 결과를 리스트에 저장
    shot_regression_results.append({
        'LOT_ID': lot_id,
        'DIE_X': die_x,
        'DIE_Y': die_y,
        'RK1': coeff_dx[0],
        'RK2': coeff_dy[0],
        'RK3': coeff_dx[1],
        'RK4': coeff_dy[1],
        'RK5': coeff_dx[2],
        'RK6': coeff_dy[2]
    })

# 회귀분석 결과를 새로운 DataFrame으로 변환
regression_df = pd.DataFrame(shot_regression_results)

# 기존 엑셀 파일을 쓰기 모드로 열고 'CPE' 시트에 회귀분석 결과를 저장
with pd.ExcelWriter("C:/vscode/MODULE/output.xlsx", engine='openpyxl', mode='a') as writer:
    regression_df.to_excel(writer, sheet_name='CPE', index=False)

print("회귀분석 결과가 엑셀 파일에 저장되었습니다.")




회귀분석 결과가 엑셀 파일에 저장되었습니다.
